In [1]:
import tensorflow as tf

tf.__version__

'2.5.0'

In [2]:
import pandas as pd

train = pd.read_csv('./input/fashion-mnist/fashion-mnist_train.csv')
test = pd.read_csv('./input/fashion-mnist/fashion-mnist_test.csv')

In [3]:
df = train.copy()

train = df.sample(frac = 0.7, random_state = 0)
valid = df.drop(train.index)

In [4]:
def split_feature_label(df):
    X = df.copy()
    y = X.pop('label')
    return X, y
    
X_train, y_train = split_feature_label(train)
X_valid, y_valid = split_feature_label(valid)
X_test, y_test = split_feature_label(test)

In [5]:
X_train.iloc[0][0:15]

pixel1       0
pixel2       0
pixel3       0
pixel4       0
pixel5       0
pixel6       0
pixel7       0
pixel8       0
pixel9       0
pixel10      0
pixel11     41
pixel12    120
pixel13    128
pixel14    127
pixel15    121
Name: 3048, dtype: int64

In [6]:
X_train /= 255.0

In [7]:
X_train.iloc[0][0:15]

pixel1     0.000000
pixel2     0.000000
pixel3     0.000000
pixel4     0.000000
pixel5     0.000000
pixel6     0.000000
pixel7     0.000000
pixel8     0.000000
pixel9     0.000000
pixel10    0.000000
pixel11    0.160784
pixel12    0.470588
pixel13    0.501961
pixel14    0.498039
pixel15    0.474510
Name: 3048, dtype: float64

In [8]:
X_train.shape[1]

784

In [9]:
X_valid /= 255.0
X_test /= 255.0

In [10]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    patience = 5,
    min_delta = 0.001,
    restore_best_weights = True,
)

In [11]:
# val_accuracy: 0.9167
"""
model = keras.Sequential([
    layers.Reshape((28, 28, 1), input_shape = (784, )),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPool2D((2, 2), strides = 2),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(10, activation = 'softmax'),
])
"""

"\nmodel = keras.Sequential([\n    layers.Reshape((28, 28, 1), input_shape = (784, )),\n    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),\n    layers.MaxPool2D((2, 2), strides = 2),\n    layers.Flatten(),\n    layers.Dense(128, activation = 'relu'),\n    layers.Dense(10, activation = 'softmax'),\n])\n"

In [12]:
from tensorflow import keras
from keras import layers

model = keras.Sequential([
    layers.Reshape((28, 28, 1), input_shape = (784, )),
    
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPool2D((2, 2), strides = 2),
    layers.Dropout(0.3),
    
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPool2D((2, 2), strides = 2),
    layers.Dropout(0.3),
    
    layers.Flatten(),
    
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.3),
    
    layers.Dense(10, activation = 'softmax'),
])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_data = (X_valid, y_valid),
    batch_size = 256,
    epochs = 500,
    callbacks = [early_stopping],
#    verbose = 0,
)

Epoch 1/500
165/165 [==============================] - 14s 83ms/step - loss: 0.7415 - accuracy: 0.7302 - val_loss: 0.4564 - val_accuracy: 0.8344
Epoch 2/500
165/165 [==============================] - 14s 84ms/step - loss: 0.4591 - accuracy: 0.8321 - val_loss: 0.3910 - val_accuracy: 0.8581
Epoch 3/500
165/165 [==============================] - 13s 81ms/step - loss: 0.4061 - accuracy: 0.8546 - val_loss: 0.3431 - val_accuracy: 0.8755
Epoch 4/500
165/165 [==============================] - 14s 82ms/step - loss: 0.3700 - accuracy: 0.8654 - val_loss: 0.3167 - val_accuracy: 0.8829
Epoch 5/500
165/165 [==============================] - 14s 87ms/step - loss: 0.3454 - accuracy: 0.8742 - val_loss: 0.3082 - val_accuracy: 0.8856
Epoch 6/500
165/165 [==============================] - 16s 96ms/step - loss: 0.3299 - accuracy: 0.8792 - val_loss: 0.2898 - val_accuracy: 0.8939
Epoch 7/500
165/165 [==============================] - 15s 93ms/step - loss: 0.3084 - accuracy: 0.8870 - val_loss: 0.2825 - val_ac

In [13]:
max(history.history['val_accuracy'])

0.92894446849823

In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_acc}")

313/313 [==============================] - 3s 9ms/step - loss: 0.1971 - accuracy: 0.9271
Test loss: 0.19712169468402863, Test accuracy: 0.9271000027656555


# Reference

[Kaggle - Intro to Deep Learning - Dropout and Batch Normalization](https://www.kaggle.com/ryanholbrook/dropout-and-batch-normalization)

[UD187 - Lession 4: Introduction to CNNs Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l04c01_image_classification_with_cnns.ipynb#scrollTo=9ODch-OFCaW4)